In [79]:
import pandas as pd
import tweepy
import numpy as np
import re
from textblob import TextBlob
# from textblob import TextBlob
# from wordcloud import WordCloud
# df = pd.read_json('output.json')
# df.to_csv('data.csv')
consumer_key = 'E4la57r5OwcofajnrdtKJTE1W'
consumer_secret = '7m9mzTwMMSEVVmEHtzK7suTUgwyEEJYdLckNoFC7husEEgTao9'
access_token = '2942662970-tUvaWqtMJCSZcXtBUXUkBIjQNsXmDV0Fv9REkQ8'
access_token_secret = 't0zfRbilLcGWDKMsOExi1MSR4AUFbA1WBRcZTL3U5xnVK'
    # twitter authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# df = pd.read_csv('data.csv') #Reading CSV file
tweetsDF = pd.read_json('output.json') #Reading json file
# df = pd.DataFrame([tweet.full_text for tweet in tweets], columns=['Tweets'])
# print (tweetsDF['full_text'])
# for index, row in tweetsDF[0:10].iterrows():
#     print(index,row['full_text'])



def cleanData(tweet):
    tweet = re.sub(r'@[A-Za-z0-9]+','', tweet) #remove @mentions
    tweet = re.sub(r'#+','', tweet) #remove # symbols
    tweet = re.sub(r'RT[\s]+','', tweet) #remove RT
    tweet = re.sub(r'https?:\/\/\S+','', tweet) #remove hyperlink
    return tweet

# apply regex on tweet to clean data
tweetsDF['full_text'] = tweetsDF['full_text'].apply(cleanData)
tweetsDF['full_text'][0:20]

    
#analysis
def getSubjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity
def getPolarity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

    # applying analysis on our text
tweetsDF['subjectivity'] = tweetsDF['full_text'].apply(getSubjectivity)
tweetsDF['polarity'] = tweetsDF['full_text'].apply(getPolarity)


#new results
tweetsDF[['full_text', 'subjectivity', 'polarity',]]

,full_text,subjectivity,polarity
0,Slappy is holding a steady floor at 100k mcap ...,0.350000,0.350000
1,"400 million, you done fucked up, shoulda divo...",0.616667,0.616667
2,Ouais je vais courir jusqu'à Marseille pour a...,0.000000,0.000000
3,: Wow.This really happened.😳 (He deserved it),0.200000,0.200000
4,What Will Smith said....\nHaileeSteinfeld Wil...,0.600000,0.600000
...,...,...,...
10879,"_OBC: ""Por gilip... puertas"":\n\n\n\nCOVID19 C...",0.000000,0.000000
10880,: fuckjoebiden fjb letsgobrandon politics poli...,0.000000,0.000000
10881,: _sanpete WillSmith,0.000000,0.000000
10882,JusticeLeague actor HarryLennix shares a state...,0.000000,0.000000
